# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Вам нужно научиться определять рыночную стоимость объектов недвижимости. Для этого проведите исследовательский анализ данных и установите параметры, влияющие на цену объектов. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта и других объектов — эти данные автоматически получены из геосервисов. Количество парков и водоёмов также заполняется без участия пользователя.

# Откройте файл с данными и изучите общую информацию

Считываю файл с сепаратором \t, вывожу информацию и первые 10 строк из файла.    
Для удобства восприятия сразу выведу информацию о данных по документации:   
- airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
- balcony — число балконов
- ceiling_height — высота потолков (м)
- cityCenters_nearest — расстояние до центра города (м)
- days_exposition — сколько дней было размещено объявление (от публикации до снятия)
- first_day_exposition — дата публикации
- floor — этаж
- floors_total — всего этажей в доме
- is_apartment — апартаменты (булев тип)
- kitchen_area — площадь кухни в квадратных метрах (м²)
- last_price — цена на момент снятия с публикации
- living_area — жилая площадь в квадратных метрах (м²)
- locality_name — название населённого пункта
- open_plan — свободная планировка (булев тип)
- parks_around3000 — число парков в радиусе 3 км
- parks_nearest — расстояние до ближайшего парка (м)
- ponds_around3000 — число водоёмов в радиусе 3 км
- ponds_nearest — расстояние до ближайшего водоёма (м)
- rooms — число комнат
- studio — квартира-студия (булев тип)
- total_area — общая площадь квартиры в квадратных метрах (м²)
- total_images — число фотографий квартиры в объявлении

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
data.info() 
print('---------------')
print(data.head(10))

In [ ]:
display(data)

По данным видим, что в таблице 23699 записей, есть столбцы с пропущенными значениями.  
Также видим несколько столбцов с типом object - is_apartment, first_day_exposition и locality_name. Для них, возможно, потребуется преобразование типов.    Сразу проверим таблицу на явные дубликаты 

In [ ]:
data.duplicated().sum()

Явных дубликатов не нашли, построим гистограммы. 

In [ ]:
data.hist(figsize=(15, 20));

Промежуточный вывод:     
1. В данных 23699 записей 
2. Есть столбцы с ненулевыми значениями   
3. Есть 3 столбца с предположительно некорректным типом данных  
4. Явных дубликатов нет   

Переходим к предобработке данных. 

# Выполните предобработку данных

На этапе предобработки данных разберёмся с пропусками, типами данных и неявными дубликатами.    
Но сначала переименуем столбец cityCenters_nearest, чтобы его название попадало под общие правила именования столбцов (snake_case, строчные буквы) 

In [ ]:
data.rename(columns = {'cityCenters_nearest':'city_centers_nearest'}, inplace = True )

Уточним количество пропусков в данных. 

In [ ]:
data.isna().sum()

Пропуски есть в следующих столбцах: 
* ceiling_height - высота потолков 
* floors_total - количество этажей 
* living_area - жилая площадь 
* is_apartment            - является ли объект апартаментами 
* kitchen_area            - площадь кухни 
* balcony                 - число балконов 
* locality_name           - название населённого пункта 
* airports_nearest        - расстояние до ближайшего аэропорта 
* city_centers_nearest     - расстояние до центра города 
* parks_around3000        - число парков в радиусе 3км 
* parks_nearest           - расстояние до ближайшего парка 
* ponds_around3000         - число водоёмов в радиусе 3км
* ponds_nearest           - расстояние до ближайшего водоёма 
* days_exposition          - сколько дней висит объявление 

Разберёмся с пропусками по очереди



## ceiling_height  
Высота потолков не указана у 9195 объектов из 23699 записей. Это почти 40% записей. 
Попробуем помимо пропусков сразу отмести ещё явно некорректные значения: в "хрущёвках" высота потолков - примерно 2.40м, предположим, что в дорогих домах премиум-класса могут быть потолки до 6 метров (квартиры с 3-метровыми потолками и "вторым светом"). Посмотрим на явно некорректные значения, выведя уникальные значения, попадающие под критерий "нереалистичных", посчитаем их количество и выведем гистограмму для "реалистичных" значений. 

In [ ]:
print('Нереалистичные значения высоты потолков: ', *data.query('ceiling_height <2.4 or ceiling_height > 6')['ceiling_height'].unique())
print('Количество таких значений - ', data.query('ceiling_height <2.4 or ceiling_height > 6')['ceiling_height'].count())
plt.hist(data['ceiling_height'], bins=50, range = (2.4,6))
plt.xlabel('Высота потолка')
plt.ylabel('Количество квартир')
plt.show()

Видим следующее: бóльшая часть значений находится в интервале 2.5-3.5 метров (и большинство значений - до трёх метров), что выглядит правдоподобно.   
Что же касается причин появления пропусков, - скорее всего, высота потолков не является обязательным значением при заполнении объявления, и пользователи просто не стали вводить их. С некорректными значениями ситуация интереснее - в некоторых значениях (например, 25.0, 27.0), похоже, пропущена запятая.   
Проставим запятые для тех строк, которые после этой операции будут попадать в наш интервал (2.40-6 метров).   
Пропущенные значения заполним медианой. 

In [ ]:
data.loc[(data['ceiling_height'] >= 24) & (data['ceiling_height'] <= 60), 'ceiling_height'] = data['ceiling_height'] / 10
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median()) 
print(data['ceiling_height'].describe())

Результат выглядит правдоподобно: средняя высота потолков в Петербурге, как и ожидалось, меньше 3 метров, но уже повыше, чем в "хрущёвках" - благодаря большому количеству новостроек у нас.   
**Общий вывод по полю**: заполнили пропуски медианным значением и подкорректировали предположительную ошибку в числе с плавающей запятой. Пропуски, скорее всего, обусловлены тем, что поле для ввода данных не обязательно к заполнению. Для более чистых данных в дальнейшем, возможно, стоит ограничить максимальные значения в этом поле, чтобы пользователи не указывали 100-метровые потолки по ошибке. 
## floors_total
В столбце floors_total пропущены значения для 86 объявлений. В целом, учитывая небольшое количество и возможности современных карт, можно было бы заполнить пропуски, исходя из адресов зданий и получить почти 100% точность - карты обновляются достаточно быстро. Но поскольку проект учебный, и пропусков не так уж много (86 из 23699 строк), можем себе позволить просто заполнить эти значения медианными.   
Для начала изучим те значения, которые всё же заполнены, чтобы выявить нереалистичные для Петербурга значения. 

In [ ]:
print(data.groupby(['floors_total'])['floors_total'].count())


Выглядит правдоподобно, теперь заполним пропуски медианным значением и проверим, не появилось ли в данных квартир, для которых номер этажа больше, чем этажность всего дома

In [ ]:
# проверим, есть ли квартиры, в которых номер этажа больше, чем этажность всего дома, в исходных данных 
print(data.query('floor > floors_total'))
data['floors_total'] = data['floors_total'].fillna(data['floors_total'].median()) 
print(data['floors_total'].describe())

In [ ]:
# проверим, есть ли квартиры, в которых номер этажа больше, чем этажность всего дома, в полученных данных
display(data.query('floor > floors_total'))
# проставим для таких случаев floors_total = floor и проверим полученные данные
data.loc[(data['floors_total'] < data['floor']), 'floors_total'] = data['floor']
display(len(data.query('floor > floors_total')))
print(data['floors_total'].describe())

Взглянем на гистограмму после заполнения пропусков и проверим, что от пропусков избавились. 

In [ ]:
plt.hist(data['floors_total'], bins=50)
plt.xlabel('Количество этажей в доме')
plt.ylabel('Количество квартир')
print('Количество пропусков в данных ',data['floors_total'].isna().sum())

Теперь, когда все пропуски заполнены, приведём тип поля floors_total к целому - нецелое количество этажей в доме кажется нереалистичным - и проверим, что тип столбца изменился. 

In [ ]:
data['floors_total'] = data['floors_total'].astype(int) 
data.info()

**Общий вывод по полю**: Заполнили пропуски медианным значением. В целом в дальнейшем, возможно, стоит научиться рассчитывать этажность дома, исходя из адреса, указанного в объявлении. Пропуски обусловлены, скорее всего, тем, что поле не обязательно к заполнению. Отдельно хочется отметить, что нецелых значений в поле не было, - скорее всего, это реализовано как ограничение на фронте при заполнении формы, но тип полученных данных при этом - float. 

## living_area
В 1903 строках пропущена жилая площадь. Изучим эти пропуски повнимательнее, посмотрим, есть может быть, жилую площадь не заполнили для каких-то конкретных квартир - например, студий? 


In [ ]:
print('Жилплощадь не заполнена для 1903 строк, доля пропусков - ', (
    data['living_area'].isna().sum()/len(data['living_area'])).round(2) * 100, 'процентов записей')
print('Из них студий -', len(data[(data['living_area'].isna()) & (data['studio'] == True)]))
print('Не студий -', len(data[(data['living_area'].isna()) & (data['studio'] == False)]))
print('Непонятно -', len(data[(data['living_area'].isna()) & (data['studio'].isna())]))
#print('Студий с незаполненной жилплощадью - ', 
#      (len(data[(data['living_area'].isna()) & (data['studio'] == True)])/len(data['studio'] == True)) * 100, 'процентов студий')
print('Процент студий среди квартир с незаполненной жилплощадью - ', 
      (len(data[(data['living_area'].isna()) & (data['studio'] == True)])/len(data[(data['living_area'].isna())]))* 100)

Похоже, студии тут ни при чём. Учитывая, что пропусков не так много, 8%, попробуем заполнить их разницей total_area и kitchen_area: значение будет приблизительное, но сильно повлиять на итог анализа не сможет. Заполним эти пропуски после обработки строк с пустой площадью кухни

## is_apartment
В целом сложно сказать, почему это поле пустует, с точки зрения разработки интерфейса я бы предположила сделать чекбокс с дефолтным значением false, в котором пустое значение было бы невозможно. Скорее всего, пользователи, которые не заполнили это значение, выставляли квартиры (апартаменты в СПб  по-прежнему не самое частое явление и в целом их продают чаще от застройщика). 
Заполним это поле значением false. 

In [ ]:
data['is_apartment'] = data['is_apartment'].fillna(value=False)
data.isna().sum()

## kitchen_area
Площадь кухни может быть не указана у студий, что логично - это подразумевает сама концепция студии. Проверим, правда ли, что для студий не указывают площадь кухни, и много ли квартир с типом "не студия" оказались без кухни. 

In [ ]:
print('Количество студий, для которых указана площадь кухни -',  
      len(data[(data['kitchen_area'].notna()) & (data['studio'] == True)]))
print('Количество студий, для которых не указана площадь кухни -',  
      len(data[(data['kitchen_area'].isna()) & (data['studio'] == True)]))
print('Количество не студий, для которых не указана площадь кухни', 
      len(data[(data['kitchen_area'].isna()) & (data['studio'] == False)]) )
print('Процент не студий среди квартир с неуказанной площадью кухни - ', 
      len(data[(data['kitchen_area'].isna()) & (data['studio'] == False)])/(len((data['studio'] == False))) * 100 )

Подтвердили предположение о том, что для студий площадь кухни не указывают. Процент нестудий среди квартир с неуказанной площадью кухни невысок, но всё же такие записи есть, их 2129. Посмотрим ещё, сколько в целом есть студий в списке, чтобы понимать, о каких числах идёт речь 

In [ ]:
print(data.groupby(['studio'])['studio'].count())

Студий не так уж много, зато с пропусками для них мы можем легко разобраться, установив площадь кухни для всех стулий равной нулю. Сразу сделаем это: 

In [ ]:
data.loc[data['studio'] == True, 'kitchen_area'] = 0
print('Количество студий, для которых не указана площадь кухни -',  
      len(data[(data['kitchen_area'].isna()) & (data['studio'] == True)]))

C кухнями в студиях разобрались. С N-комнатными квартирами ситуация посложнее: тут придётся площадь кухни "придумать". Можно предположить, что площадь кухни может зависеть от "комнатности" квартиры, поэтому можно заполнить площадь кухни для таких квартир медианными значениями в разрезе по количеству комнат в квартире. Сделаем это таким образом: для каждого количества комнат посчитаем отношение площади кухни к общей площади квартиры, возьмём медианное значение этого отношения (например, площадь кухни - половина всей квартиры для "однушек") и заполним площадь кухни, исходя из полученной медианы и общей площади квартиры. Для прозрачности добавим вывод в цикле 

In [ ]:
for i in range(1, data['rooms'].max()):
    area_median = data[data['rooms'] == i]['kitchen_area'].median()
    ratio_median = (data[data['rooms'] == i]['kitchen_area']/data[data['rooms'] == i]['total_area']).median()
    print(f'Медианная площадь кухни для {i} комнат - ', area_median, ', медианное отношение', ratio_median)
    data['kitchen_area'] = data['kitchen_area'].fillna(value = data['total_area'] * ratio_median)
    
#сразу выведу информацию о том, что поменялось 
print(data['kitchen_area'].describe())
print(data.info())

Проверим важный момент: в данных нет пропусков в общей площади квартиры. Значит, мы точно можем узнать, не оказалась ли площадь кухни больше, чем площадь всей квартиры. 

In [ ]:
print('Количество квартир, где площадь кухни оказалась больше всей площади ', len(data[data['kitchen_area'] > data['total_area']]))


Теперь, когда пропуски в площади кухни заполнены, попробуем заполнить жилую площадь.

In [ ]:
data['living_area'] = data['living_area'].fillna(value = (data['total_area'] - data['kitchen_area']))
data.isna().sum()

Заполнили пропуски в площади кухни и заодно в жилой площади. 

## balcony
Посмотрим общую информацию по столбцу balcony - какие там значения. 

In [ ]:
print(data.groupby(['balcony'])['balcony'].count())
print('Пустых значений - ', len(data[data['balcony'].isna()]))

По опыту заполнения объявлений и заявок можно предположить, что количество балконов не заполняется в случае, когда балконов просто нет. Заполним пропущенные значения нулями и приведём значение поля к целочисленному формату - количество балконов - это обычно целое число. 

In [ ]:
data['balcony'] = data['balcony'].fillna(0)
data['balcony']=data['balcony'].astype('int64')
print(data.groupby(['balcony'])['balcony'].count())
print('Пустых значений - ', len(data[data['balcony'].isna()]))

## locality_name
В столбце не так много пропущенных значений, и проще всего просто расстаться с этими строками. Уберём строки с незаполненным городом из нашего набора данных, их процент незначителен для анализа. 

In [ ]:
print('Пустых значений - ', len(data[data['locality_name'].isna()]), 
      'Процент пустых значений - ', 
      len(data[data['locality_name'].isna()])/len(data['locality_name'])*100)
data.dropna(subset=['locality_name'], inplace=True)
print('Пустых значений - ', len(data[data['locality_name'].isna()]), 
      'Процент пустых значений - ', 
      len(data[data['locality_name'].isna()])/len(data['locality_name'])*100)

## airports_nearest

Данное значение восстановить будет тяжело, поскольку нужно проверять адреса домов и высчитывать более 5 тысяч значений вручную. 
Что касается причин пропусков - либо сервис объявлений предлагает заполнять это значение пользователям самостоятельно, либо рассчитывает на основе адреса по встроенным картам. Пока ничего с пропусками сделать не можем, оставим их. 

## city_centers_nearest, parks_around3000, ponds_around3000, parks_nearest, ponds_nearest
Ситуация с расстоянием до центра похожая, либо сервис предлагает пользователям вбивать это расстояние вручную, либо рассчитывает на основе адреса. Либо 5 тысяч пользователей не стали вводить эти данные, либо при подсчёте расстояния возникла ошибка. 
С такими пропусками мы ничего сделать не можем, усреднять расстояние будет не совсем корректно, оставим эти пропуски. 

Аналогичная ситуация и с расстоянием до парков и прудов и количеством парков и прудов в районе 3 км: у нас нет сервиса для обработки картографических данных, поэтому самостоятельно на основе адреса мы эти данные не проставим. 

Уичитывая, что данные поля мы в дальнейшем будем использовать для исследования, проставим им "фейковое" значение -1 = таким образом, мы сразу поймём, что тут был пропуск, при этом не будем вынуждены считать расстояние исходя из адреса, и к тому же, сделаем поле непустым. 

In [ ]:
data['city_centers_nearest'] = data['city_centers_nearest'].fillna(-1)
data['parks_around3000'] = data['parks_around3000'].fillna(-1)
data['ponds_around3000'] = data['ponds_around3000'].fillna(-1)
data['parks_nearest'] = data['parks_nearest'].fillna(-1)
data['ponds_nearest'] = data['ponds_nearest'].fillna(-1)

## days_exposition
Пропуски в этом поле кажутся очень странными, поскольку скорее всего, это системные данные, которые пользователь заполнять не должен. Возможно, данные пропущены здесь в старых объявлениях, которые заводились до апгрейда системы, после которого система начала считать этот показатель, или же банально произошла ошибка при выгрузке/сохранении данных. В любом случае, с этими пропусками мы ничего сделать сейчас не можем, оставим их. 

## Остальные преобразования
Помимо работы со столбцами, где есть пропуски, обратим внимание и на те столбцы, где пропусков нет. 
### Столбец last_price 
Мы имеем дело с продажей квартир. Это означает, что речь идёт о больших суммах, и вряд ли стоимость с точностью до копейки для кого-то будет принципиальной 

In [ ]:
data[['last_price']] = data[['last_price']].apply(lambda x: x.astype('int64'))
#data['last_price'] = data.astype({'last_price': int})
# проверяем
data.info()
data['last_price'].describe()

Переведём дату первой выкладки объявления в формат даты, выкинем время публикации, поскольку для дальнейшего анализа оно не требуется в рамках нашей задачи, и переименуем столбец, чтобы с ним в дальнейшем было проще работать 

In [ ]:
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')
data['first_day_exposition'] = data['first_day_exposition'].dt.round('1d') # отбросим время
data = data.rename(columns={'first_day_exposition' : 'date'}) # переименуем столбец для краткости
# проверяем
data.dtypes

Посмотрим на оставшиеся пропуски

In [ ]:
data.isna().sum()

print('----')

data.info()

### locality_name 
Изучим столбец locality_name на предмет дубликатов, ошибок, опечаток и так далее. 

In [ ]:
print(data['locality_name'].sort_values().unique())
print(len(data['locality_name'].unique()))

Сразу бросается в глаза проблема с заполнением "е" и "ё", здесь, к сожалению, придётся ё заменить на е. К сожалению, это частая беда, люди просто не пишут ё.

In [ ]:
data['locality_name'] = data['locality_name'].str.replace('ё','е')
print(data['locality_name'].sort_values().unique())
print(len(data['locality_name'].unique()))

Немного сократили количество названий. Попробуем найти дубликаты в названиях, убрав тип населённого пункта (посёлок, деревня и так далее). 

In [ ]:
def shorten_locality(text):
    dict =['деревня ', 
           'коттеджный поселок ', 'коттеджный посёлок', 'садовое товарищество ', 
           'поселок городского типа ', 'поселок при железнодорожной станции ', 'поселок станции ', 
           'городской поселок ', 'городской посёлок', 'садоводческое некоммерческое товарищество '
           'посёлок городского типа ', 'поселок имени', 'поселок ', 'посёлок', 'село ']
    for i in dict:
        text = text.replace(i, '')  
    return text   
# занесём данные в отдельный столбец - на всякий случай, чтобы не потерять данные 
data['locality_name_shortened'] = data['locality_name'].apply(shorten_locality)
# посмотрим разницу по столбцам 
print(len(data['locality_name'].unique()))
print(len(data['locality_name_shortened'].unique()))

# Предобработка данных - вывод 
Выявлены пропуски в различных столбцах и заполнены, где это было возможно: 
- ceiling_height: заполнили пропуски медианным значением и подкорректировали предположительную ошибку в числе с плавающей запятой. Пропуски, скорее всего, обусловлены тем, что поле для ввода данных не обязательно к заполнению. Для более чистых данных в дальнейшем, возможно, стоит ограничить максимальные значения в этом поле, чтобы пользователи не указывали 100-метровые потолки по ошибке.

##### floors total: 
Заполнили пропуски медианным значением. В целом в дальнейшем, возможно, стоит научиться рассчитывать этажность дома, исходя из адреса, указанного в объявлении. Пропуски обусловлены, скорее всего, тем, что поле не обязательно к заполнению. Отдельно хочется отметить, что нецелых значений в поле не было, - скорее всего, это реализовано как ограничение на фронте при заполнении формы, но тип полученных данных при этом - float. Преобразовали поле в int. 

##### is_apartment
Предположительно пропуски в этом поле обусловлены тем, что пользователям предлагается заполнить его. Учитывая небольшой процент апартаментов в городе и относительно редкую частоту их продажи не от застройщика, предполагаем, что для объявлений с незаполненным значением этого поля будет верно проставить False. 
В дальнейшем можно порекомендовать сделать это поле чекбоксом в объявлениях с значением False по умолчанию 

##### kitchen_area 
Может быть не заполнено по разным причинам - как ошибка в выгрузке, так и пользователь не знает/не захотел заполнять/поленился заполнять. Заполнили это значение медианным в разрезе по количеству комнат в квартирах. Для студий указали нулевую площадь кухонь (потому что студия предполагает кухню, не отделённую от всей площади)

##### balcony 
Здесь пропуски скорее всего обусловлены человеческим фактором, пользователи просто не тсали заполнять информацию о количестве балконов. Предполагаем, что информация не заполняется в большинстве случаев из-за того, что балконов просто нет. Заполнили пропуски нулями. 

##### locality_name 
Небольшое количество пропущенных здесь значений можно обусловить технической ошибкой, возникшей при сохранении или выгрузке данных. Сложно представить, что это поле могло быть необязательным в сервисе размещения объявлений. Пропусков не так много, удалили строки с ними. 

##### airports_nearest, city_centers_nearest, parks_around3000? ponds_around3000? parks_nearest, ponds_nearest
Пропуски в этих полях можно обусловить тремя разными причинами: 
- системная ошибка при расчёте этой информации, если в систему встроен алгоритм расчёта этих данных на основе адреса 
- человеческий фактор (кто-то действительно заполнил эти данные сам, кто-то пропустил)
- появление этих данных в новой версии системы - в этом случае "старые" объявления ещё не содержали этой информации. 
Данные из этих полей не так критичны для нашего исследования, и пропуски в полях оставлены. 

##### days_exposition 
Пропуски в этом поле похожи на системную ошибку. У нас нет информации о дате закрытия объявления или хотя бы о статусе объявления (открыто-закрыто), поэтому сделать мы с ними ничего не сможем, поэтому оставили их. 

Привели данные в столбцах к более удобным для нас типам, данные готовы к дальнейшей работе.

# Добавьте в таблицу новые столбцы

В рамках проекта требуется модифицировать и обогатить наши данные. 
Нужно добавить столбцы: 
1. цена одного квадратного метра (нужно поделить стоимость объекта на его общую площадь, а затем округлить до двух знаков после запятой);
1. день недели публикации объявления (0 — понедельник, 1 — вторник и так далее);
1. месяц публикации объявления;
1. год публикации объявления;
1. тип этажа квартиры (значения — «‎первый», «последний», «другой»);
1. расстояние до центра города в километрах (переведите из м в км и округлите до ближайших целых значений).

In [ ]:
# добавляем цену квадратного метра 
data['price_square_meter'] = (data['last_price'] / data['total_area']).round(2)

In [ ]:
# Для задания требуется столбец с номером дня недели, но мне, возможно, будет в дальнейшем удобнее 
# работать с текстовым названием дня недели. Добавлю отдельным столбцом текстовое отображение дня недели 
def weekday_type(row):
    if row['weekday']==0:
        return 'понедельник'
    if row['weekday']==1:
        return 'вторник'
    if row['weekday']==2:
        return 'среда'
    if row['weekday']==3:
        return 'четверг'
    if row['weekday']==4:
        return 'пятница'
    if row['weekday']==5:
        return 'суббота'
    if row['weekday']==6:
        return 'воскресенье'
    return "Ошибка"

# добавляем номер дня недели 
data['weekday'] = data['date'].dt.weekday
data['weekday_text']=data.apply(weekday_type, axis=1)

In [ ]:
# добавляем месяц публикации 
data['month'] = data['date'].dt.month

In [ ]:
# добавляем год отдельной колонкой 
data['year'] = data['date'].dt.year

In [ ]:
# добавляем тип этажа = первый, последний или другой, для этого сначала везде выставим "другой",
# а затем проапдейтим данные для первых и последних этажей 
data['floor_type'] = 'другой'
data.loc[data['floor'] == 1, 'floor_type'] = 'первый'
data.loc[data['floor'] == data['floors_total'], 'floor_type'] = 'последний'

In [ ]:
# расстояние до центра города в километрах - при этом надо округлить до целых значений. Делим на 1к и округляем 
data['city_center_km'] = (data['city_centers_nearest'] / 1000).round(0)
#выведем первые 5 строк 
print(data.head(5))

## Выводы 
Рассчитали данные и добавили требуемые столбцы. Упростили отображение номера дня недели для удобства в дальнейшем  

# Проведите исследовательский анализ данных
Начнём с изучения указанных в задании параметров по порядку. 
Требуется изучить следующее: 
- построить гистограммы для рассматриваемых полей
- рассмотреть и удалить редкие и выбивающиеся значения
- восстановить корректные значения, если понятна природа аномалии
- если непонятна природа аномалии, выкинуть данные

Информация нужа по следующим полям: 
- общая площадь;
- жилая площадь;
- площадь кухни;
- цена объекта;
- количество комнат;
- высота потолков;
- тип этажа квартиры («первый», «последний», «другой»);
- общее количество этажей в доме;
- расстояние до центра города в метрах;
- расстояние до ближайшего парка

Рассмотрим их по порядку - для начала выведем общую информацию и построим гистограммы. 
###### общая площадь
Нам потребуется выводить много гистограмм и диаграмм размаха в рамках анализа, попробуем это автоматизировать, чтобы вводить меньше параметров

In [ ]:
def draw_hist(col, xmin, xmax):
    data[col].hist(bins=100, range=(xmin, xmax), figsize=(15,5))
def draw_boxplot(col, ymin=-50, ymax=200):
    plt.ylim(ymin, ymax)
    data.boxplot(col)
#после определения функций выведем на всякий случай общую информацию по общей площади
data['total_area'].describe()

In [ ]:
#отдельно выделим "нижние" 5 процентов и "верхние" 5 процентов и 1 процент - смущает максимальная площадь в 900 квадратов при вполне ожидаемой медиане и среднем 
data.total_area.describe(percentiles=[0.05, 0.5, 0.95, 0.99]) # общие данные

В целом значения можно назвать ожидаемыми, медиана в 60 квадратных метров и среднее в 52 квадрата - медиана достаточно близка к среднему, из чего можно сделать вывод, что выбросы не так критичны. 
Построим разные гистограммы - по всему интервалу, по основной массе и по верхним 5% квартир - посмотрим, какой разброс там.

In [ ]:
# гистограмма по всему интервалу
#data.hist('total_area', bins=100, figsize=(15,5)) 
draw_hist('total_area', 12, 900)
plt.title('Распределение площадей всех квартир')
plt.show()
# гистограмма по основной массе - 90% всех квартир
draw_hist('total_area', 31, 116)
plt.title('Распределение площадей по основной массе - 90% всех квартир')
plt.show()
# гистограмма по верхним 5% квартир
draw_hist('total_area', 116, 900)
plt.title('Распределение площадей по верхним 5% квартир')
plt.show()

In [ ]:
# очень любопытно, что это за квартиры такие с огромными площадями, посмотрим на них
print(data[data['total_area'] >= 600])
# и сразу построим диаграмму размаха, чтобы увидеть аномальные значения и убрать их, как того требует заданеи
draw_boxplot('total_area')

Аномальными можно считать значения до 20 метров и от 120. Уберём их, хотя это, конечно и очень любопытные для Петербурга данные

In [ ]:
data = data.query('total_area >= 20 and total_area <= 120')
draw_hist('total_area', 12, 900)
plt.title('Распределение площадей всех квартир после обновления таблицы')
plt.show()

In [ ]:
data['living_area'].describe()

In [ ]:
draw_hist('living_area', 2, 101)
plt.title('Распределение жилпощади')
plt.show()
draw_boxplot('living_area')

In [ ]:
# посмотрим, что же это за квартиры такие с жилой площадью меньше 10 квадратов - может, какая-то ошибка, и пропущена точка? сравним с тем, что показывает нам общая площадь 
print(data.query('living_area < 10 and total_area > 50'))

Вывод по полю: Не хотелось верить в квартиры с жилплощадью менее хотя бы 10 квадратов, но беглый поиск по циану показал, что можно купить студию площадью с жилой площадью 2-3 метра, а вот общая площадь квартир при этом сильно варьируется. 
Возможно, желание удалить квартиры с такой маленькой жилплощадью субьективно, поэтому оставим эти строки в выборке.  

В целом в нашей выборке больше всего квартир небольшой жилой площади, от 10 до 30 квадратов, при этом есть довольно значительный спад в этом диапазоне в районе 25 квадратов. 



###### площадь кухни
Опять же, не самый показательный параметр, по нему будет крайне сложно делать выводы - на рынке попадаются квартиры и без кухни, и апартаменты с площадью в полтора и даже полметра, и к тому же, мы сами заполняли площадь кухни нулями для студий, но всё же изучим параметр и построим хотя бы диаграммы по нему. 

In [ ]:
data['kitchen_area'].describe()

In [ ]:
draw_hist('kitchen_area', 0, 64)
plt.title('Распределение площади кухни')
plt.show()
draw_boxplot('kitchen_area')

In [ ]:
# посмотрим поближе гистограмму по "популярным" значнеиям 
draw_hist('kitchen_area', 4, 12)
plt.title('Распределение площади кухни')
plt.show()
# посмотрю, что же это за кухни такие больше нынешних двушек
print(data.query('kitchen_area > 50'))

Вывод по полю: и "аномально" большие кухни, и маленькие выглядят правдоподобно в своих объявлениях - это либо студии, либо квартиры с большой общей площадью и малой жилплощадью, чаще всего расположенные довольно далеко от центра. Удалять их не будем. Интересно, что в объявлениях прослеживается тенденция на указание площади кухни как целого числа, это видно по гистограмме по "популярным" значениям и легко объясняется тем, что площадь кухни мало кто в принципе обычно знает до десятых или сотых долей. 

###### цена объекта
Смотрим общие данные, строим гистограмму и диаграмму размаха 

In [ ]:
data['last_price'].describe()

In [ ]:
#для чтения в human-readable формате выведем цены немного по-другому 
#pd.set_eng_float_format(accuracy=1, use_eng_prefix=True) 
# этот параметр был включен из любопытства, а теперь не выключается :
#pd.options.display.float_format = '{:20,.2f}'.format
data['last_price'].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
draw_hist('last_price', 1.219000e+04, 8.200000e+07 )
plt.title('Распределение всех цен')
plt.show()
draw_hist('last_price', 2.219000e+04, 8.200000e+06 )
plt.title('Распределение относительно часто встречающихся цен')
plt.show()
draw_hist('last_price', 11.800000e+06, 8.200000e+08 )
plt.title('Распределение верхних 5%')
plt.show()

Получается, пик цен - от 3 до 5 миллионов, квартир дороже 20 и даже 10 миллионов существенно меньше. 


In [ ]:
# смотрим на квартиры дороже 18 миллионов
print(data[data['last_price'] > 18000000])
# смотрим на квартиры дешевле полумиллиона 
print(data[data['last_price'] < 500000])

In [ ]:
# избавляемся от нереалистичных цен на жильё - менее 250 тысяч рублей 
data = data.query('last_price > 250000')
# проверяем итог 
print(data[data['last_price'] < 250000])

Самые высокие цены аномальными назвать сложно, они кажутся реалистичными, их убирать пока не будем. 
Что касается нереалистично низких цен, можно было бы предположить, что в некоторых объявлениях продавец ошибся с запятой (особенно в случае с квартирой площадью больше 100 квадратов и стоимостью 12 тысяч рублей), но тут можно ошибиться с порядком. Кроме того, продавец мог вообще выставить цену в другой валюте, не посмотрев, что требутеся в объявлении. Поэтому эти данные отбросили. 
Промежуточный вывод по полю -  пик цен - от 3 до 5 миллионов, квартир дороже 20 и даже 10 миллионов существенно меньше. 

###### количество комнат
Изучим общую информацию по полю, а затем построим гистограмму 

In [ ]:
data['rooms'].describe()


In [ ]:
draw_hist('rooms', 0, 7)

Есть квартиры с нулём комнат - это нормально, у нас есть в выборке студии, и крайне мало квартир с количеством комнат больше 6. Посмотрим на них повнимательнее

In [ ]:
print(data.query('rooms > 6'))

Одна квартира, стоимостью 13 миллионов, 7 комнат, 100 квадратов. Цена не слишком реалистичная для нашего времени, но возможно, для 2018 года данные правдоподобны. Сложно назвать эту строку аномалией, оставляем её. 

Общий вывод по полю: ожидаемо медианное значение количества комнат - 2, в большинстве квартир 1-2 комнаты, трёхкомнатных квартир чуть меньше, и есть квартиры-"гиганты" (в частности, бывшие коммуналки, характерные для нашего города), которых значительно меньше. Аномальных данных не найдено. 

###### высота потолков
Сразу посмотрим общую информацию по полю и посмотрим гистограмму. 

In [ ]:
data['ceiling_height'].describe()

In [ ]:
draw_hist('ceiling_height', 1, 5)

Сразу выкинули из выборки 100-метровые потолки, чтобы посмотреть на бОльшую часть данных на гистограмме поближе.   
Картина реалистичная - средняя высота потолков от 2.5 метров до 3, есть некоторое количество элитных или старых зданий/квартир с "вторым светом". 

Посмотрим поближе на квартиры с аномально высокими потолками, особенно интересует аномальное здание с 100-метровыми потолками

In [ ]:
print(data.query('ceiling_height > 5'))

Сложно объяснить 100-метровые потолки в однушке на последнем этаже с площадью в 25 метров. Уберём это значение и посмотрим на итог без него. Учитывая, что в среднем высота потолка получается между 2.5 и 3 метрами, сложно привести указанные в полученных 4 строках значения к чему-то адекватному. Выкинем эти данные и посмотрим на итог 

In [ ]:
data = data.query('ceiling_height < 5')
data['ceiling_height'].describe()

Изучим также аномально низкие потолки, чтобы понять, можно что-то исправить или просто выкинем эти данные. 

In [ ]:
print(data[data['ceiling_height'] < 2.2])

14 строк, объявления разных лет, разные дома - откажемся от этих данных, удалив строки, и посмотрим на общее описание и гистограмму после удаления.

In [ ]:
data = data.query('ceiling_height > 2.2')
data['ceiling_height'].describe()

In [ ]:
draw_hist('ceiling_height', 2.25, 4.8)

Пик значений находится на 2.7.  
Общий вывод по полю: удалили аномально низкие и высокие значения, в целом видим, что большинство квартир имеет потолки высотой в районе 2.68 метров 

###### тип этажа квартиры («первый», «последний», «другой»)
Снова посмотрим на общее описание поля и построим гистограмму. В качестве общего описания поля выведем просто количество строк с каждым из значений - медиана и среднее нас тут не будет интересовать. 

In [ ]:
data.groupby('floor_type').count()

In [ ]:
draw_hist('floor_type', 'другой', 'последний')

Здесь сложно счесть какие-либо данные аномальными, это этаж, на котором расположена квартира. Заметим, что количество квартир на первом и последнем этажах примерно одинаково, квратир на остальных этажах ожидаемо больше. 

Общий вывод по полю: распределение значений ожидаемое, аномалий не найдено, отбрасывать данные не будем. 

###### общее количество этажей в доме
Выведем общую информацию и построим гистограмму. Выведем значения, которые показались аномально большими

In [ ]:
data['floors_total'].describe()

In [ ]:
draw_hist('floors_total', 1, 60)

In [ ]:
print(data[data['floors_total'] > 37])

Поиск утверждает, что самые высокие жилые дома в СПб достигают 37 этажей. Выкинем данные с 60 и 52 этажами как нереалистичные. 

In [ ]:
data = data.query('floors_total < 37')
data['floors_total'].describe()

Общий вывод по полю: отбросили нереалистичные значения с большим количеством этажей. В целом по данным видим вполне закономерные пики на 5 этажах (максимальная этажность жилого дома без необходимости строить лифты) и 9 этажах (максимальная этажность, при которой можно держать всего один лифт в парадной). Ожидала бы ещё пик на 16 и 25 этажах, но в нашей выборке не так много высокоэтажных домов, поэтому такие данные не так заметны. 

###### расстояние до центра города в метрах
Снова смотрим общее описание и строим гистограмму, рассматриваем значения, которые показались аномальными. Работать будем с полем city_center_km, которое мы заводили для сохранения информации о расстоянии в километрах

In [ ]:
# выведем данные без учёта "заглушек" в расстоянии до центра города, где были проставлены значения "-1"
display(data.query('city_center_km > 0')['city_center_km'].describe())

In [ ]:
draw_hist('city_center_km', 0,66)

In [ ]:
draw_boxplot('city_center_km', 0, 70)

In [ ]:
# посмотрим на строки с расстоянием от центра больше 40 км. Для Санкт-Петербурга это уже сильно большое значение - в 40 км от центра уже начинается Гатчина. 
display(data.query('city_center_km > 40'))

Данные не выглядят реалистично, и их не очень много - пожертвуем этими строками

In [ ]:
data = data.query('city_center_km < 40')
display(data.query('city_center_km > 0')['city_center_km'].describe())
draw_hist('city_center_km', 0, 40)
plt.title('Гистограмма после удаления нереалистичных данных')
plt.show()

Общий вывод по полю: убрали нереалистичные значения свыше 40 км от центра города. В среднем видим, что продаются квартиры в основном в районе 5-17 км от центра, что похоже на правду, если посмотреть на карту города. 

###### расстояние до ближайшего парка
Проделаем все те же операции с этим полем: построим гистограмму, посмотрим на значения, посмотрим на аномальные значения, если такие есть, уберём явно аномальные строки.

In [ ]:
data[data['parks_nearest'] > 0]['parks_nearest'].describe()

In [ ]:
draw_hist('parks_nearest', 1,3190)

**Общий вывод по полю**: Петербург сложно назвать "зелёным" городом, у нас много районов, где действительно нет парков, поэтому даже 3190 метров до ближайшего парка не выглядит нереалистичным значением. 
В целом наиболее частое расстояние до парка - чуть меньше 500 метров, сильно удалённые от парков квартиры встречаются, но сложно назвать это аномалией, зная город. 

**Общий вывод по разделу**: рассмотрели гистограммы по предложенным полям, отбросили аномальные значения там, где это требуется, подчистили данные от аномалий. Данные можно считать готовыми для решения основной задачи - дальнейшего исслпдования. 

В целом продают в квартиры площадью около 60 квадратов, в среднем жилая площадь - 31 метр. В среднем размер кухни - 10 метров, при этом продавцы имеют тенденцию указывать площадь кухни как целое число, игнорируя десятые и сотые доли. 

Цена квартир закономерно имеет некоторый разброс, но наибольшее количество объявлений предлагает квартиры от 3 до 5 миллионов. В большинстве своём это однокомнатные и двухкомнатные квартиры. 

Средняя высота потолка - 2.68, что в принципе бьётся со стандартами. 

Большинство квартир продаётся в пяти- и девятиэтажных домах на расстоянии от центра примерно в 5-15 километров (в спальных районах), при этом расстояние до ближайшего парка в целом невелико - от 300 до 700 метров в среднем. 


Приступим к разделу с исследованием данных


## исследование скорости продажи квартир
Изучите, как быстро продавались квартиры (столбец days_exposition). Этот параметр показывает, сколько дней было размещено каждое объявление.
Постройте гистограмму.
Посчитайте среднее и медиану.
В ячейке типа markdown опишите, сколько времени обычно занимает продажа. Какие продажи можно считать быстрыми, а какие — необычно долгими?

Для начала изучим в целом информацию по тому, как долго висели объявления. 

In [ ]:
data['days_exposition'].describe()

In [ ]:
draw_hist('days_exposition', 0, 1572)

In [ ]:
# построим гистограмму на наиболее "популярных" значениях 
draw_hist('days_exposition', 0, 221)

**Общий вывод по полю**: среднее значение времени продажи квартиры - 213 дней, медианное - 175. Медианное не зависит от выбросов, поэтому возьмём его за "стандартное" среднее. Сразу определимся, что такое "быстрые" продажи - это "нижние" 25 процентов, то есть 44 дня, и "долгие" продажи - "верхние" 75 процентов, то есть 223 дня. 

В целом, по опыту, объявления, которые висят так долго, покупатель может счесть неактуальными, кроме того, обычно более "старые" объявления системы отправляют в конец списка. Я бы сказала, что "долгие" продажи рискуют затянуться, если владелец не будет обновлять объявление. Для проектировщика системы было бы полезно внедрить функциональность с напоминанием об актуализации объявлений, если её ещё нет, чтобы помочь пользователям продавать жильё быстрее, а платформе - работать эффективнее. 


## определение факторов, влияющих на скорость продажи 
Определите факторы, которые больше всего влияют на общую (полную) стоимость объекта.
Изучите, зависит ли цена от:
- общей площади;
- жилой площади;
- площади кухни;
- количества комнат;
- этажа, на котором расположена квартира (первый, последний, другой);
- даты размещения (день недели, месяц, год).

Постройте графики, которые покажут зависимость цены от указанных выше параметров. Для подготовки данных перед визуализацией вы можете использовать сводные таблицы.

Итак, нужно изучить корреляцию цены (last_price) и указанных полей 
 - общая площадь;
 - жилая площадь;
 - площадь кухни;
 - количество комнат;
 - тип этажа, на котором расположена квартира (первый, последний, другой);
 - дата размещения (день недели, месяц, год).
  
 Изучим этот вопрос. Добавим к списку параметров новые столбцы, которые мы добавляли в рамках проекта (день недели, месяц и год публикации объявления)


In [ ]:
# соберу список параметров в переменную, чтобы ничего не потерять и не скроллить постоянно на текст задания 
impact_parameters = [
    'last_price',
    'total_area',
    'living_area',
    'kitchen_area',
    'rooms']
# считаю корреляцию сразу "всего со всем"
data_corr = data[impact_parameters].corr()
# Выведем на экран
display(data_corr)


In [ ]:
# построим красивый heatmap, добавлю сразу annot = true, чтобы показать числовые значения 
import seaborn
plt.figure(figsize = (15, 10))
seaborn.heatmap(data_corr, vmin=-1, vmax=1,
            cmap='coolwarm', annot=True)
plt.title('Матрица корреляции')
plt.show()

Видим следующее: есть умеренная положительная связь 
- между ценой и общей площадью
- между ценой и жилой площадью 
- между ценой и площадью кухни 
- между ценой и количеством комнат. 

Столбец floor_type придётся рассмотреть отдельно. 

Дополнительно - видим хорошую корреляцию между количеством комнат и общей/жилой площадью, что ещё раз говорит, что наши отфильтрованные данные достаточно правдоподобны. 
Посмотрим графики для вышеуказанных полей, для которых "нашлась" корреляция и для floor_type

In [ ]:
def draw_plot_scatter(x, y):
    (
        data.plot(x = x, y = y, kind = 'scatter')
    )
    plt.title(f'Зависимость цены от {x}')
    plt.show()
draw_plot_scatter('total_area', 'last_price');
draw_plot_scatter('living_area', 'last_price');
draw_plot_scatter('kitchen_area', 'last_price');

In [ ]:
# для floor_type и rooms добавим линии в график 
def draw_plot_lines(col):
    (
        data.pivot_table(index=col, values='last_price')
        .plot(grid=True, style='o-', figsize=(5, 5))
    )
    plt.title(f'Зависимость цены от {col}')
    plt.show()

draw_plot_lines('rooms')
draw_plot_lines('floor_type')
draw_plot_lines('weekday')
draw_plot_lines('month')
draw_plot_lines('year')

Видим, что квартиры на первом этаже дешевле, чем остальные; квартиры на последнем этаже в среднем чуть дешевле, чем на остальных, но здесь разница не так значительна. 

**Общий вывод по блоку**: 

Есть зависимость

- между ценой группой полей, показывающих "размер" квартиры - 
    - общей площадью
    - жилой площадью
    - количеством комнат
    - площадью кухни 
- между ценой и типом этажа (квартиры на первом этаже в среднем дешевле, квартиры на последнем этаже немного дешевле). 

Также помимо заметной корреляции можем отметить следующее: 
- в объявлениях, выставленных зимой-в начале весны, цена в среднем выше 
- с 2017 года цены начали расти 

Любопытно, что  в объявлениях, выставленных с понедельника по четверг, цена в среднем выше, чем в объявлениях, выставленных в пятницу и в выходные. 

## средняя цена квадратного метра в 10 городах 
Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений — постройте сводную таблицу с количеством объявлений и средней ценой квадратного метра для этих населенных пунктов. 

Выделите населённые пункты с самой высокой и низкой стоимостью квадратного метра.  

Выбираем 10 населённых пунктов с наибольшим количеством объявлений. На всякий случай построим данные и по "старому" столбцу, и по "новому" - с сокращёнными названиями

In [ ]:
# выбираем населённые пункты с наибольшим числом объявлений. выведу топ 15 просто чтобы посмотреть, каким населённым пунктам не хватило объявлений до попадания в топ 
print(data.groupby(['locality_name'])['locality_name'].count().sort_values(ascending=False).head(15))
print(data.groupby(['locality_name_shortened'])['locality_name_shortened'].count().sort_values(ascending=False).head(15))

Топ 10 выглядит более чем ожидаемо: сам Петербург, Мурино и Шушары - по сути части города, и далее самые "популярные" города. Ожидала увидеть Гатчину повыше, но возможно, субьективные ожидания не попадают на годы сбора данных. Состав топ 10 не меняется при использовании "укороченных" названий, меняется только порядок. В дальнейшем будем работать с "укороченным" названием. 

Создаём таблицу со стоимостью квадратного метра по этим городам. Чтобы не перечислять их по названиям, просто составим таблицу по всем данным и выведем топ 10 

In [ ]:
# создаём таблицу с городом, ценой и полсчётом количества объявлений 
top_price_square_meter = data.pivot_table(index='locality_name_shortened', values='last_price', aggfunc=['count'])
top_price_square_meter.columns = ['count']
#сортируем 
top_price_square_meter = top_price_square_meter.sort_values(by='count', ascending=False)
#  отдельно заводим таблицу для топ 10
top10_price_square_meter = top_price_square_meter.sort_values(by='count', ascending=False).head(10)
print(top10_price_square_meter)
# добавляем столбец с ценой квадратного метра и сортируем по этому столбцу
top10_price_square_meter['price_square_meter'] = data.pivot_table(index='locality_name_shortened', values='price_square_meter').round(2)
print(top10_price_square_meter.sort_values(by='price_square_meter', ascending=False))

In [ ]:
# выведем ещё красивый пайчарт по этим городам 
top10_price_square_meter.plot(y='count', kind='pie', figsize=(7, 7));

Поскольку в задании есть необходимость вывести населённый пункт с самой высокой и низкой стоимостью квадратного метра, но не сказано, что среди топ 10 городов, выведем город с самой высокой низкой стоимостью квадрата в принципе. 

In [ ]:
top_price_square_meter['price_square_meter'] = data.pivot_table(index='locality_name_shortened', values='price_square_meter').round(2)
print(top_price_square_meter.sort_values(by='price_square_meter', ascending=True).head(3))
print(top_price_square_meter.sort_values(by='price_square_meter', ascending=False).head(3))

Общий вывод по разделу: Создали отдельную таблицу с количеством объявлений и ценой квадратного метра - для всех городов и для городов с наибольшим количеством объявлений (топ-10). 
- самый дорогой квадратный метр среди "популярных" городов - в Санкт-Петербурге 
- самый дешёвый квадратный метр среди городов "топ 10" - в Выборге. 
- самый дорогой квадртаный метр в принципе среди всех объявлений оказался в Кронштадте. 
- самый дешёвый квадратный метр среди всех объявлений оказался в Старополье. 

## стоимость квартир на разном удалении от центра
Ранее вы посчитали расстояние до центра в километрах. Теперь выделите квартиры в Санкт-Петербурге с помощью столбца locality_name и вычислите их среднюю стоимость на разном удалении от центра. Учитывайте каждый километр расстояния: узнайте среднюю цену квартир в одном километре от центра, в двух и так далее. Опишите, как стоимость объектов зависит от расстояния до центра города — постройте график изменения средней цены для каждого километра от центра Петербурга.

In [ ]:
# выделим квартиры в Санкт-Петербурге 
# data.info()
data.query('locality_name == "Санкт-Петербург"')['city_center_km'].describe()
# создадим таблицу 
(
    data.query('locality_name == "Санкт-Петербург"')
    .pivot_table(index='city_center_km', values='last_price')
    .plot(grid=True, style='o-', xlim=(0,25), figsize=(10, 5))
)
plt.title('Зависимость цены от расстояния до центра')
plt.xlabel('Расстояние до центра города (километр)')
plt.ylabel('Цена')
plt.show()
print('Зависимость средней стоимости от расстояния до центра')
print(data.query('locality_name == "Санкт-Петербург"').pivot_table(values='last_price', index='city_center_km').head(20))

**Общий вывод по разделу**: Квартиры ближе к центру ожидаемо стоят дороже, наблюдается тенденция к снижению цены при увеличении расстояния до центра. После 9 километров от центра цена в целом уже меняется не так значительно. В целом судя по ситуаици с недвижимостью картина выглядит логично, хотя с практической точки зрения для жилья привлекательнее "спальные" районы Петербурга. 

# Напишите общий вывод

В рамках проекта исследованы данные об объявлениях о продаже недвижимости.   
**В рамках обработки и подготовки данных**
- Явных дубликатов в данных не найдено, пропуски в данных сочтены некритичными и заполнены там, где это было возможно (преимущественно медианными значениями) 
- Произведены преобразования некоторых полей по типу данных (количество этажей, этаж не могут быть нецелыми) 
- Сделаны вывод о причинах пропусков в данных, среди возможных причин 
    - человеческий фактор (пользователи не заполняют некоторые значения)
    - технический сбой 
    - несоответствие версий фронта и бэка или ввод данных в систему в тот момент, когда производилось обновление до новой версии
- даны рекомендации по расширению функциональности системы, чтобы избежать подобных проблем в дальнейшем: 
    - система расчёта расстояния до парков и центра города, исходя из адреса 
    - система расчёта количества этажей в доме на основе адреса 
    - единый справочник населённых пунктов 
- Для анализа данных добавлены новые "технические" столбцы и таблицы для визуализации. 

**В рамках исследовательского анализа данных**
- Изучены основные параметры, построены гистограммы, сделаны выводы о выбросах, отброшены "аномальные" данные 
- Изучена скорость продажи квартир, выявлены параметры "быстрой" и "долгой" продажи: 
    - среднее значение времени продажи квартиры - 213 дней, медианное - 175. Медианное не зависит от выбросов, поэтому возьмём его за "стандартное" среднее. Сразу определимся, что такое "быстрые" продажи - это "нижние" 25 процентов, то есть 44 дня, и "долгие" продажи - "верхние" 75 процентов, то есть 223 дня
- Изучены параметры, влияющие на стоимость квартиры: найдена зависимость
    - между ценой группой полей, показывающих "размер" квартиры -
        - общей площадью
        - жилой площадью
        - количеством комнат
        - площадью кухни
    - между ценой и типом этажа (квартиры на первом этаже в среднем дешевле, квартиры на последнем этаже немного дешевле).
- Посчитана стоимость квадратного метра для "топ 10" городов в объявлениях: 
    - самый дорогой квадратный метр среди "популярных" городов - в Санкт-Петербурге
    - самый дешёвый квадратный метр среди городов "топ 10" - в Выборге.
    - самый дорогой квадртаный метр в принципе среди всех объявлений оказался в Кронштадте.
    - самый дешёвый квадратный метр среди всех объявлений оказался в Старополье.
- Посчитана средняя стоимость квартир в зависимости отрасстояния до центра и выявлена зависимость стоимости от расстояния до центра города 
    - Квартиры ближе к центру ожидаемо стоят дороже, наблюдается тенденция к снижению цены при увеличении расстояния до центра. После 9 километров от центра цена в целом уже меняется не так значительно.

**Чек-лист готовности проекта**

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Файл с данными открыт.
- [x]  Файл с данными изучен: выведены первые строки, использован метод `info()`, построены гистограммы.
- [x]  Найдены пропущенные значения.
- [x]  Пропущенные значения заполнены там, где это возможно.
- [x]  Объяснено, какие пропущенные значения обнаружены.
- [x]  В каждом столбце установлен корректный тип данных.
- [x]  Объяснено, в каких столбцах изменён тип данных и почему.
- [x]  Устранены неявные дубликаты в названиях населённых пунктов.
- [x]  Обработаны редкие и выбивающиеся значения (аномалии).
- [x]  В таблицу добавлены новые параметры:
       – цена одного квадратного метра;
       – день публикации объявления (0 - понедельник, 1 - вторник и т. д.);
       – месяц публикации объявления;
       – год публикации объявления;
       – тип этажа квартиры (значения — «первый», «последний», «другой»);
       – расстояние до центра города в километрах.
- [x]  Изучены и описаны параметры:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - цена объекта;
        - количество комнат;
        - высота потолков;
        - тип этажа квартиры («первый», «последний», «другой»);
        - общее количество этажей в доме;
        - расстояние до центра города в метрах;
        - расстояние до ближайшего парка.
- [x]  Выполнено задание «Изучите, как быстро продавались квартиры (столбец `days_exposition`)»:
    - построена гистограмма;
    - рассчитаны среднее и медиана;
    - описано, сколько обычно занимает продажа и указано, какие продажи можно считать быстрыми, а какие — необычно долгими.
- [x]  Выполнено задание «Определите факторы, которые больше всего влияют на общую (полную) стоимость объекта». Построены графики, которые показывают зависимость цены от параметров:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - количество комнат;
        - тип этажа, на котором расположена квартира (первый, последний, другой);
        - дата размещения (день недели, месяц, год).
- [x]  Выполнено задание «Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений»:
    - выделены населённые пункты с самой высокой и низкой стоимостью квадратного метра.
- [x]  Выполнено задание «Выделите квартиры в Санкт-Петербурге с помощью столбца `locality_name` и вычислите их среднюю стоимость на разном удалении от центра»:
    -  учтён каждый километр расстояния, известны средние цены квартир в одном километре от центра, в двух и так далее;
    -  описано, как стоимость объекта зависит от расстояния до центра города;
    -  построен график изменения средней цены для каждого километра от центра Петербурга.
- [x]  На каждом этапе сделаны промежуточные выводы.
- [x]  В конце проекта сделан общий вывод.